auth.getSession

Fetch a session key for a user. The third step in the authentication process. See the authentication how-to for more information.
Params

token (Required) : A 32-character ASCII hexadecimal MD5 hash returned by step 1 of the authentication process (following the granting of permissions to the application by the user)
api_key (Required) : A Last.fm API key.
api_sig (Required) : A Last.fm method signature. See authentication for more information.

artist.getTopTracks

Get the top tracks by an artist on Last.fm, ordered by popularity
Example URLs

JSON: /2.0/?method=artist.gettoptracks&artist=cher&api_key=YOUR_API_KEY&format=json

XML: /2.0/?method=artist.gettoptracks&artist=cher&api_key=YOUR_API_KEY
Params

artist (Required (unless mbid)] : The artist name
mbid (Optional) : The musicbrainz id for the artist
autocorrect[0|1] (Optional) : Transform misspelled artist names into correct artist names, returning the correct version instead. The corrected artist name will be returned in the response.
page (Optional) : The page number to fetch. Defaults to first page.
limit (Optional) : The number of results to fetch per page. Defaults to 50.
api_key (Required) : A Last.fm API key.

PASO 1: Obtener token de autenticación que necesitaremos para iniciar sesión.

In [2]:
import requests
import pandas as pd

API_KEY = '357e10dfd8c3d43a08795cbb92a9cf24'
API_SECRET = '1702885e042cee4d290d834146c2891a'

url = f'https://ws.audioscrobbler.com/2.0/?method=auth.gettoken&api_key={API_KEY}&format=json'
params = {
    'api_key': API_KEY
}

response = requests.get(url, params=params)

if response.status_code == 200:
    token_value = response.text.split(':')[1].strip('}"')  # Guarda el token de la respuesta
    print(f'Tu token es: {token_value}')
else:
    print('Error al obtener el token')

/Users/fisanchezrafferty/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Tu token es: TcZ_oYeAh-767mugEph2tURfAD3u2WVQ


PASO 2: Autorizar el token en la WEB de last.fm.com -- > https://www.last.fm/api/auth?api_key=INTRODUCIR_API_KEY&token=INTRODUCIR_TOKEN:
https://www.last.fm/api/auth?api_key=357e10dfd8c3d43a08795cbb92a9cf24&token=INTRODUCIR_TOKEN

PASO 3: Obtener sesión.
Concatenar los parámetros en el orden correcto para generar el api_sig. El api_sig es una firma que asegura que la solicitud es válida. Se genera concatenando los parámetros en este orden: api_key, method, token, y api_secret.

In [113]:
import requests
import hashlib

# Paso 1: Concatenar los parámetros en el orden correcto para generar el api_sig
concatenated_params1 = f'api_key{API_KEY}methodauth.getSessiontoken{token_value}{API_SECRET}'
api_sig = hashlib.md5(concatenated_params1.encode('utf-8')).hexdigest()

# Paso 2: Preparar los parámetros para la solicitud POST
url3 = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'auth.getSession',
    'api_key': API_KEY,
    'token': token_value,
    'api_sig': api_sig,
    'format': 'json'
}

# Hacer la solicitud POST con los parámetros en el cuerpo de la solicitud
response = requests.post(url3, data=params)

# Comprobación de la respuesta
if response.status_code == 200:
    print('La llamada es correcta')
    session_key = response.json()['session']['key']  # Mostrar la respuesta en formato JSON
    print(session_key)
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar más detalles del error si es posible

La llamada es correcta
K9yTrrMFEzcpYl6jYwPcgy8u7hLFOGlZ


In [118]:
import datetime
import time

timestamp_actual = int(time.time())
#fecha_y_hora = datetime.datetime(2023, 10, 22, 12, 0, 0)
#timestamp = int(fecha_y_hora.timestamp())

track = 'Feel Good Inc.'
artist = 'Gorillaz'

url = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'track.scrobble',
    'artist': artist,
    'track': track,
    'timestamp': timestamp_actual,
    'api_key': API_KEY,
    'api_sig': api_sig,
    'sk': session_key,
    'format': 'json'
}

# Hacer la solicitud POST para scrobblear la canción
response = requests.post(url, params=params)

# Comprobar la respuesta
if response.status_code == 200:
    print('Canción scrobbleada correctamente.')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')

Error 400: Bad Request


In [39]:
url1 = f'http://www.last.fm/api/auth/?{API_KEY}token={token}'
params = {
    'token': token,
    'api_key': API_KEY,
    #'api_sig': API_SECRET
    }

response = requests.get(url1, params=params)

if response.status_code == 200:
    print(f'la llamada es correcta')
else:
    print(response.status_code)
    print(response.reason)

ConnectionError: HTTPSConnectionPool(host='www.last.fm', port=443): Max retries exceeded with url: /api/auth/?357e10dfd8c3d43a08795cbb92a9cf24token=%7B%22token%22:%22l2lkO9D3hVl0gYN9Z096uUvqddhjT6YZ%22%7D&token=%7B%22token%22%3A%22l2lkO9D3hVl0gYN9Z096uUvqddhjT6YZ%22%7D&api_key=357e10dfd8c3d43a08795cbb92a9cf24 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x129f42ac0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [33]:
import hashlib

Error 403: Forbidden


In [ ]:
# Paso 1: Concatenar los parámetros en el orden correcto
concatenated_params = f'api_key{API_KEY}methodauth.getSessiontoken{token}{API_SECRET}'

# Paso 2: Generar el hash MD5 de la cadena concatenada
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Paso 3: Hacer la solicitud con los parámetros correctos
url2 = f'https://ws.audioscrobbler.com/2.0/?method=auth.getSession&api_key={API_KEY}&token={token}&api_sig={api_sig}&format=json'

response = requests.get(url2)

# Comprobación de la respuesta
if response.status_code == 200:
    print('La llamada es correcta')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')

In [40]:
# Paso 1: Concatenar los parámetros en el orden correcto para generar el api_sig
concatenated_params1 = f'api_key{API_KEY}methodauth.getSessiontoken{token}{API_SECRET}'

# Paso 2: Generar el hash MD5 de la cadena concatenada
api_sig = hashlib.md5(concatenated_params1.encode('utf-8')).hexdigest()

# Paso 3: Preparar los parámetros para la solicitud POST
url3 = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'auth.getSession',
    'api_key': API_KEY,
    'token': token,
    'api_sig': api_sig,
    'format': 'json'
}

# Hacer la solicitud POST con los parámetros en el cuerpo de la solicitud
response = requests.post(url3, data=params)

# Comprobación de la respuesta
if response.status_code == 200:
    print('La llamada es correcta')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar más detalles del error si es posible

Error 403: Forbidden
{"message":"Unauthorized Token - This token has not been issued","error":4}


PRUEBA CON CÓDIGO UNIDO 

In [111]:
import requests
import hashlib
import datetime

# Definición de variables
API_KEY = 'YOUR_API_KEY'  # Reemplaza con tu API Key
API_SECRET = 'YOUR_API_SECRET'  # Reemplaza con tu API Secret
session_key = 'YOUR_SESSION_KEY'  # Reemplaza con tu SESSION_KEY

# Definir la fecha y hora para el timestamp
fecha_y_hora = datetime.datetime(2023, 10, 22, 12, 0, 0)
timestamp = int(fecha_y_hora.timestamp())

# Definir la canción y el artista
track = 'Waka Waka'
artist = 'Shakira'

# Generar api_sig
concatenated_params = (
    f'api_key{API_KEY}'
    f'artist{artist}'
    f'methodtrack.scrobble'
    f'sk{session_key}'
    f'timestamp{timestamp}'
    f'track{track}'
    f'{API_SECRET}'
)
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Preparar los parámetros para la solicitud
url = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'track.scrobble',
    'artist': artist,
    'track': track,
    'timestamp': timestamp,
    'api_key': API_KEY,
    'api_sig': api_sig,
    'sk': session_key,
    'format': 'json'
}

# Hacer la solicitud POST para scrobblear la canción
response = requests.post(url, data=params)  # Usar `data` en lugar de `params`

# Comprobar la respuesta
if response.status_code == 200:
    print('Canción scrobbleada correctamente.')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar más detalles del error

Error 403: Forbidden
{"message":"Invalid API key - You must be granted a valid key by last.fm","error":10}


PRUEBA

In [119]:
import requests
import hashlib
import time

# Obtener el timestamp actual
timestamp_actual = int(time.time())

# Información de la canción


# Generar api_sig
concatenated_params = (
    f'api_key{API_KEY}'
    f'artist{artist}'
    f'methodtrack.scrobble'
    f'sk{session_key}'
    f'timestamp{timestamp_actual}'
    f'track{track}'
    f'{API_SECRET}'  # La API_SECRET se concatena al final
)
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Preparar los parámetros para la solicitud POST
url = 'https://ws.audioscrobbler.com/2.0/'
params = {
    'method': 'track.scrobble',
    'artist': artist,
    'track': track,
    'timestamp': timestamp_actual,
    'api_key': API_KEY,
    'api_sig': api_sig,
    'sk': session_key,
    'format': 'json'
}

# Hacer la solicitud POST para scrobblear la canción
response = requests.post(url, data=params)  # Usar `data` para enviar los parámetros en el cuerpo

# Comprobar la respuesta
if response.status_code == 200:
    print('Canción scrobbleada correctamente.')
    print(response.json())  # Mostrar la respuesta en formato JSON
else:
    print(f'Error {response.status_code}: {response.reason}')
    print(response.text)  # Mostrar detalles adicionales del error

Canción scrobbleada correctamente.
{'scrobbles': {'scrobble': {'artist': {'corrected': '0', '#text': 'Gorillaz'}, 'album': {'corrected': '0'}, 'track': {'corrected': '0', '#text': 'Feel Good Inc.'}, 'ignoredMessage': {'code': '0', '#text': ''}, 'albumArtist': {'corrected': '0', '#text': ''}, 'timestamp': '1729718465'}, '@attr': {'ignored': 0, 'accepted': 1}}}


In [125]:
rs = response.json()
rs

{'scrobbles': {'scrobble': {'artist': {'corrected': '0', '#text': 'Gorillaz'},
   'album': {'corrected': '0'},
   'track': {'corrected': '0', '#text': 'Feel Good Inc.'},
   'ignoredMessage': {'code': '0', '#text': ''},
   'albumArtist': {'corrected': '0', '#text': ''},
   'timestamp': '1729718465'},
  '@attr': {'ignored': 0, 'accepted': 1}}}

In [3]:
url_base = 'https://ws.audioscrobbler.com/2.0'
params_info = {
        'method': 'artist.getinfo',
        'artist': 'Shakira',
        'api_key': API_KEY,
        'format': 'json'
        }
artista = requests.get(url_base, params=params_info)

In [4]:
artista.status_code

200

In [5]:
artista_json = artista.json()

Artista.

Biografía del Artista.

Listeners (cantidad de gente que lo escucha).

Playcount (cantidad de veces reproducido su música)

Artistas Similares.

In [152]:
artista_json['artist']

{'name': 'Shakira',
 'mbid': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
 'url': 'https://www.last.fm/music/Shakira',
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'extralarge'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'mega'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': ''}],
 'streamable': '0',
 'ontour': '1',
 'stats': {'listeners': '4000724', 'playcount': '129982799'},
 'similar': {'artist': [{'name': 'Paulina Rubio',
    'url': 'https://www.l

In [143]:
artista = artista_json['artist']['name']

'Shakira'

In [151]:
biografia_artista = artista_json['artist']['bio']['content']

'Shakira Isabel Mebarak Ripoll is a Colombian singer. Born and raised in Barranquilla, She made her recording debut with Sony Music Colombia at the age of 13. Following the commercial failure of her first two albums, Magia (1991) and Peligro (1993), she rose to prominence in Hispanic countries with her next albums, Pies Descalzos (1995) and Dónde Están los Ladrones? (1998). She entered the English-language market with her fifth album, Laundry Service (2001), which sold over 13 million copies worldwide. Buoyed by the international success of her singles "Whenever, Wherever" and "Underneath Your Clothes", the album propelled her reputation as a leading crossover artist. Broadcast Music, Inc., described Shakira as a "pioneer" who extended the global reach of Latino singers.\n\nHer success was further solidified with the Spanish albums Fijación Oral, Vol. 1 (2005), Sale el Sol (2010) and El Dorado (2017), all of which topped the Billboard Top Latin Albums chart and were certified diamond (

In [153]:
artista_json['artist']['stats']

{'listeners': '4000724', 'playcount': '129982799'}

In [154]:
listeners = artista_json['artist']['stats']['listeners']

'4000724'

In [155]:
playcount = artista_json['artist']['stats']['playcount']

'129982799'

In [168]:
artista_json['artist']['similar']['artist']

lista_artistas_similares = artista_json['artist']['similar']['artist']

lista_nombres_artistas = []

for elemento in lista_artistas_similares:
    lista_nombres_artistas.append(elemento['name'])
print(lista_nombres_artistas)


['Paulina Rubio', 'Karol G', 'Belinda', 'Thalía', 'Julieta Venegas']


In [ ]:
dic_lastfm = {"artista": [], 'biografia_artista': [], 'listeners': [], 'playcount': [], 'artistas_similares': [] }

lista_artistas_buscar = {}

In [29]:
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv 

import csv
import json

In [30]:
# Paso 1: Leer el archivo CSV
spoti_csv_file = "../data/dataframes_combinados.csv" # nombre de tu archivo Spoti CSV
spoti_json_file = "dataframes_combinados.json"  # nombre del archivo Spoti de salida JSON

# Paso 2: Abrir el CSV y convertir a formato JSON
with open(spoti_csv_file, mode='r', encoding='utf-8') as file:
    # Leer el contenido CSV
    csv_reader = csv.DictReader(file)
    
    # Convertir cada fila a diccionario y agregarla a una lista
    data = [row for row in csv_reader]

# Paso 3: Escribir los datos en un archivo JSON
with open(spoti_json_file, mode='w', encoding='utf-8') as file:
    json.dump(data, file, indent=4)  # Guarda el JSON con formato legible (indent=4)

print(f'Archivo {spoti_json_file} creado con éxito.')

Archivo dataframes_combinados.json creado con éxito.


In [31]:
#Traemos el archivo a json
with open("dataframes_combinados.json","r") as f: 
    result_spoti = json.load(f)

In [32]:
result_spoti

[{'nombre_cancion_album': 'Seventeen',
  'artista': 'Ladytron',
  'tipo_album_cancion': 'album',
  'año_lanzamiento': '2022',
  'género': 'Electrónica'},
 {'nombre_cancion_album': 'Crust',
  'artista': 'Flying Lotus',
  'tipo_album_cancion': 'album',
  'año_lanzamiento': '2021',
  'género': 'Electrónica'},
 {'nombre_cancion_album': 'Abu Simbel',
  'artista': 'WhoMadeWho',
  'tipo_album_cancion': 'single',
  'año_lanzamiento': '2021',
  'género': 'Electrónica'},
 {'nombre_cancion_album': 'Apricots',
  'artista': 'BICEP',
  'tipo_album_cancion': 'album',
  'año_lanzamiento': '2021',
  'género': 'Electrónica'},
 {'nombre_cancion_album': 'KILL DEM',
  'artista': 'Jamie xx',
  'tipo_album_cancion': 'single',
  'año_lanzamiento': '2022',
  'género': 'Electrónica'},
 {'nombre_cancion_album': 'Water',
  'artista': 'BICEP',
  'tipo_album_cancion': 'single',
  'año_lanzamiento': '2022',
  'género': 'Electrónica'},
 {'nombre_cancion_album': 'Polyghost',
  'artista': 'Bonobo',
  'tipo_album_cancio

In [176]:
artist = []

for item in result_spoti: 
    artist.append(item['artista'])

In [180]:
#Convertimos en set para eliminar artistas duplicados (hay artistas con mas de una canción en el mismo año)
set_artist = set(artist)
len(set_artist)

893

In [ ]:
def llamar_api(url):
    
    llamada = requests.get(url_base)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada.json()  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.

In [17]:
artista_json_key = artista_json['artist'][]

In [18]:
artista_json_key

{'name': 'Shakira',
 'mbid': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
 'url': 'https://www.last.fm/music/Shakira',
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'extralarge'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'mega'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': ''}],
 'streamable': '0',
 'ontour': '1',
 'stats': {'listeners': '4001737', 'playcount': '130040023'},
 'similar': {'artist': [{'name': 'Paulina Rubio',
    'url': 'https://www.l

In [89]:
dict_shakira = {"artista": [], 'biografia_artista': [], 'listeners': [], 'playcount': [], 'artistas_similares': []}

try:
    dict_shakira['artista'].append(artista_json_key['name'])
except(KeyError):
    dict_shakira['artista'].append("NULL")

print(dict_shakira)



{'artista': ['Shakira'], 'biografia_artista': [], 'listeners': [], 'playcount': [], 'artistas_similares': []}


In [27]:
for elemento in artista_json_key:
    dict(elemento)
    

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [41]:
for elemento in result_spoti:
    print(elemento)

{'nombre_cancion_album': 'Seventeen', 'artista': 'Ladytron', 'tipo_album_cancion': 'album', 'año_lanzamiento': '2022', 'género': 'Electrónica'}
{'nombre_cancion_album': 'Crust', 'artista': 'Flying Lotus', 'tipo_album_cancion': 'album', 'año_lanzamiento': '2021', 'género': 'Electrónica'}
{'nombre_cancion_album': 'Abu Simbel', 'artista': 'WhoMadeWho', 'tipo_album_cancion': 'single', 'año_lanzamiento': '2021', 'género': 'Electrónica'}
{'nombre_cancion_album': 'Apricots', 'artista': 'BICEP', 'tipo_album_cancion': 'album', 'año_lanzamiento': '2021', 'género': 'Electrónica'}
{'nombre_cancion_album': 'KILL DEM', 'artista': 'Jamie xx', 'tipo_album_cancion': 'single', 'año_lanzamiento': '2022', 'género': 'Electrónica'}
{'nombre_cancion_album': 'Water', 'artista': 'BICEP', 'tipo_album_cancion': 'single', 'año_lanzamiento': '2022', 'género': 'Electrónica'}
{'nombre_cancion_album': 'Polyghost', 'artista': 'Bonobo', 'tipo_album_cancion': 'album', 'año_lanzamiento': '2022', 'género': 'Electrónica'}


In [45]:
dic_lastfm = {"artista": []}

for elemento in result_spoti:

    try:
        dic_lastfm["artista"].append(elemento['artista'])
    except(KeyError):
        "El nombre del artista no está en la lista"
    
    try: 
        

    #dic_lastfm.append(elemento['artist']['name'])
    
print(dic_lastfm)

{'artista': ['Ladytron', 'Flying Lotus', 'WhoMadeWho', 'BICEP', 'Jamie xx', 'BICEP', 'Bonobo', 'Bonobo', 'Bonobo', 'Flying Lotus', 'BICEP', 'Bonobo', 'Bonobo', 'Metronomy', 'Flying Lotus', 'Burial', 'LCD Soundsystem', 'Joy Orbison', 'Flying Lotus', 'Bonobo', 'Rhye', 'Justice', 'Weval', 'Röyksopp', 'Justice', 'Metronomy', 'Jamie xx', 'Ladytron', 'Peces Raros', 'Rhye', 'Nightmares On Wax', 'Ladytron', 'Nosaj Thing', 'BICEP', 'Röyksopp', 'Ross from Friends', 'Christian Löffler', 'LOUTA', 'Flying Lotus', 'SBTRKT', 'WhoMadeWho', 'Metronomy', 'Ross from Friends', 'Daphni', 'Metronomy', 'Metronomy', 'Bonobo', 'Bonobo', 'Clark', 'Caribou', 'Flying Lotus', 'Bonobo', 'Clark', 'Warpaint', 'Jamie xx', 'Romare', 'Röyksopp', 'Bonobo', 'Floating Points', 'Moderat', 'Flying Lotus', 'Morcheeba', 'Tycho', 'Faithless', 'WhoMadeWho', 'Rhye', 'Mogwai', 'Joy Orbison', 'Bonobo', 'Ladytron', 'BICEP', 'Ross from Friends', 'Portico Quartet', 'DJ Koze', 'Björk', 'Röyksopp', 'Warpaint', 'Rhye', 'Max Cooper', 'Bon

In [65]:
url_base = 'https://ws.audioscrobbler.com/2.0'
params_info = {
        'method': 'artist.getinfo',
        'artist':  'shakira',
        'api_key': API_KEY,
        'format': 'json'
        }
artista = requests.get(url_base, params=params_info)

In [66]:
artista

<Response [200]>

In [68]:
a_json = artista.json()

In [73]:
a_json

{'artist': {'name': 'Shakira',
  'mbid': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
  'url': 'https://www.last.fm/music/Shakira',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '1',
  'stats': {'listeners': '4001737', 'playcount': '130040023'},
  'similar': {'artist': [{'name': 'Paulina Rubi

In [78]:
a_json['artist']['name']

'Shakira'

In [85]:
for e in a_json['artist']['stats']['listeners']:
    print(e)

4
0
0
1
7
3
7


In [178]:
url_base = 'https://ws.audioscrobbler.com/2.0'
params_info = {
        'method': 'artist.getinfo',
        'artist':  'shakira, gorillaz',
        'api_key': API_KEY,
        'format': 'json'
        }
artista = requests.get(url_base, params=params_info)

for elemento in artista_json:

    try:
        a_json.get(['artist']['name'])
    except(KeyError):
        "El nombre del artista no está en la lista"
    
    try:
        elemento





dic_lastfm = {"artista": [], 'biografia_artista': [], 'listeners': [], 'playcount': [], 'artistas_similares': []}





In [179]:
artista.status_code

414